# 4.1 - Intro Unsupervised Learning

**Ejemplos:** (Cuando no hay etiquetas)

+ Segmentación de clientes
+ Segmentación de productos
+ One-person segment (lo que funciona bien en un grupo no funciona necesariamente en otro)
+ Agrupación según características

**Escenario**

Cada elemento (usuario, cliente, producto..) puede ser considerado un vector. Dado un paquete de vectores se distribuyen por clusters. Vectores similares estarán en el mismo cluster, y clusters similares estarán más 'cerca' entre si que los que son más diferentes.

Todo éste proceder asume la existencia de una métrica de distancia, así se puede medir dicha distancia entre vectores dentro del espacio vectorial.

**Pasos para aplicar USL:**

+ **1**: Representación vectorial
+ **2**: Métrica, función de distancia
+ **3**: Medida de similitud

Cuando se crea un cluster, el vector en el centro del cluster (centroide) se considera el prototipo o el más representativo del cluster.


**Problemas tratables por USL:**

+ Segmentación de mercado
+ Detección de fraude en seguros o banca
+ Respuesta a tratamientos médicos
+ Detección de outliers

**Ejemplos de modelos:**

+ K-Means
+ Hierarchical Clustering
+ DBSCAN
+ HDBSCAN
+ GMM

# K-Means


El más viejo y el más popular. La idea es seleccionar previamente cuantos clusters se desean ($k$). Se seleccionan los puntos centrales del cluster (centroides) de manera aleatoria. Para cada nuevo registro se reasigna un cluster y se recalcula el centroide (media). Se itera el proceso hasta que no haya cambio en la clusterización.

![kmeans](images/kmeans.png)

![kmeans2](images/kmeans2.png)


https://www.naftaliharris.com/blog/

https://www.naftaliharris.com/blog/visualizing-k-means-clustering/

In [ ]:
import pylab as plt
%matplotlib inline

import pandas as pd
import numpy as np

In [ ]:
census=pd.read_csv('../data/acs2015_county_data.csv')

census=census.dropna()

census.head()

In [ ]:
census.info()

In [ ]:
cols=[c for c in census.columns.values if c not in ['CensusId', 'State', 'County']]

cols[:10]

In [ ]:
# KMeans 

from sklearn.cluster import KMeans

In [ ]:
kmeans = KMeans(n_clusters=4)

In [ ]:
clusters = kmeans.fit(census[cols])

In [ ]:
clusters.cluster_centers_.shape

In [ ]:
clusters.cluster_centers_[0]

In [ ]:
dict(zip(cols, clusters.cluster_centers_[0]))

In [ ]:
preds = clusters.fit_predict(census[cols])

preds[300:400]

In [ ]:
preds.shape

In [ ]:
census[cols].shape

In [ ]:
distancias = clusters.fit_transform(census[cols])

distancias.shape

In [ ]:
distancias[0]

In [ ]:
preds[310]

In [ ]:
distancias[np.where(preds==2)][0]

In [ ]:
census['cluster'] = preds

census.head()

In [ ]:
census.cluster.value_counts()

In [ ]:
census.columns

In [ ]:
census.groupby('cluster')['Income'].mean()  # media de ingresos por cluster

In [ ]:
census[census.cluster==1].head()

In [ ]:
census[census.cluster==1]

In [ ]:
census.groupby('cluster')['TotalPop'].mean()   # media de poblacion por cluster

In [ ]:
census.groupby('cluster')['Employed'].mean()   # media de empleo por cluster

##### plot 2D K-Means

In [ ]:
np.set_printoptions(precision=5, suppress=True)

np.random.seed(4711)

a=np.random.multivariate_normal([10,0], [[3,1], [1,4]], size=[100,])

b=np.random.multivariate_normal([0,20], [[3,1], [1,4]], size=[50,])

In [ ]:
X=np.concatenate((a,b),)
X.shape

In [ ]:
plt.scatter(X[:,0], X[:,1])
plt.show();

In [ ]:
kmeans=KMeans(n_clusters=2)

clusters=kmeans.fit(X)

In [ ]:
plt.scatter(X[:,0], X[:,1], c=clusters.fit_predict(X))

plt.scatter([clusters.cluster_centers_[i][0] for i in range(2)],
            [clusters.cluster_centers_[i][1] for i in range(2)], 
            c='r')

plt.show();

# Hierarchical Clustering (Dendrograma)

La mejora sobre K-Means es fundamentalmente que no hay que asumir el número de clusters.

Hay dos maneras de atacar éste problema:

+ Por aglomeración
+ Por división

**Aglomeración:**

Se empieza asumiendo que cada registro es un cluster individual y se van combinando aquellos similares hasta conseguir un solo cluster, la población entera. Después se decide el número óptimo de clusters según la distancia.


**División:**

El proceso inverso. Se comienza con un solo cluster y se va dividiendo hasta tener el número óptimo de clusters (según distancia).


![hca](images/hca.png)

##### Medidas de distancia

Aquí también entra la distancia, para medir entre registros/clusters. Normalmente, se usa la distancia euclídea. El uso de la métrica de distancia depende del problema en cuestión, por ejemplo, en una clusterización de lugares en la ciudad, podría ser mejor usar $cityblock$, o incluso tomar el tiempo que hay entre dos lugares (tiempo como métrica de distancia).

##### Criterios de conexión (linkage criteria)

Después de seleccionar la métrica de distancia, hay que decidir de qué manera se conectan los registros al cluster. Puede ser:

+ Single-linkage: conexión entre los puntos más parecidos del cluster.
+ Complete-linkage: conexión entre los puntos menos parecidos del cluster.
+ Average-linkage: conexión entre los centroides del cluster.


![link](images/link.png)

Al igual que la métrica de distancia, la elección del criterio de conexión ha de basarse en el propio problema. Por ejemplo, para clasificar varias tecnologías diferentes tendría sentido basarse en cuales son más parecidos. Si no existen justificaciones claras de cual usar, se suele usar el criterio de Ward, que viene a ser mínimos cuadrados de las distancias, respecto de la media del cluster. Esto concuerda con el concepto de distancia entre grupos en estadística (ANOVA, MANOVA....)

https://es.wikipedia.org/wiki/M%C3%A9todo_de_Ward

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage

In [ ]:
#help(linkage)

In [ ]:
# matriz de conexion

Z=linkage(X, 'ward')

Z.shape

In [ ]:
X.shape

In [ ]:
Z[3] # idx1, idx2, distancia, count

In [ ]:
idx=[48, 100]

plt.scatter(X[:, 0], X[:, 1])

plt.scatter(X[idx, 0], X[idx, 1], c='r');

##### dendrograma

In [ ]:
plt.figure(figsize=(15, 8))

dendrogram(Z)

plt.show();

In [ ]:
dendrogram(Z, truncate_mode='lastp', p=12,
           leaf_rotation=90., leaf_font_size=8.,
           show_contracted=True, show_leaf_counts=False)

plt.title('Dendrograma')
plt.xlabel('indice de muestra')
plt.ylabel('distancia')

plt.show();

In [ ]:
def fancy_dendro(*args, **kwargs):
    
    max_d=kwargs.pop('max_d', None)
    
    if max_d and 'color_threshold' not in kwargs:
        kwargs['color_threshold']=max_d
        
    annotate_above=kwargs.pop('annotate_above', None)
    
    ddata=dendrogram(*args, **kwargs)
    
    if not kwargs.get('no_plot', False):
        plt.title('Dendrograma')
        plt.xlabel('Indice de muestra o (tamaño del cluster)')
        plt.ylabel('Distancia')
        
        for i,d,c in zip(ddata['icoord'], ddata['dcoord'], ddata['color_list']):
            x=0.5*sum(i[1:3])
            y=d[1]
            
            if y>annotate_above:
                plt.plot(x,y,'o', c=c)
                plt.annotate('%.3g'%y, (x,y), xytext=(0,-5),textcoords='offset points', va='top', ha='center')
                
        if max_d:
            plt.axhline(y=max_d, c='k')
    
    return ddata
    
    

In [ ]:
fancy_dendro(Z, truncate_mode='lastp', p=12,
                 leaf_rotation=90., leaf_font_size=8.,
                 show_contracted=True, annotate_above=10)

plt.show();

In [ ]:
fancy_dendro(Z, truncate_mode='lastp', p=12,
                 leaf_rotation=90., leaf_font_size=8.,
                 show_contracted=True, annotate_above=10, 
             
             max_d=50)

plt.show();

##### obteniendo clusters

In [ ]:
from scipy.cluster.hierarchy import fcluster

In [ ]:
max_d=20  # por distancia

clusters=fcluster(Z, max_d, criterion='distance')

clusters

In [ ]:
k=2   # por el nº de clusters

clusters=fcluster(Z, k, criterion='maxclust')

clusters

In [ ]:
plt.scatter(X[:,0], X[:,1], c=clusters)

plt.show();

# K-Means VS Hierarchical

In [ ]:
from sklearn.datasets import make_moons

In [ ]:
moons=make_moons(n_samples=100, shuffle=True, noise=0.05)[0]

plt.scatter(moons[:,0], moons[:,1]);

In [ ]:
from sklearn.cluster import KMeans

from sklearn.cluster import AgglomerativeClustering as AC

In [ ]:
cluster=KMeans(n_clusters=4).fit(moons)

plt.figure(figsize=(10, 6))

plt.scatter(moons[:, 0], moons[:, 1], c=cluster.labels_)

plt.scatter([cluster.cluster_centers_[i][0] for i in range(4)],
            [cluster.cluster_centers_[i][1] for i in range(4)],
            c='r')

;

In [ ]:
cluster=AC().fit(moons)     # 2 clusters , ward por defecto

plt.figure(figsize=(10, 6))

plt.scatter(moons[:, 0], moons[:, 1], c=cluster.labels_);

In [ ]:
cluster=AC(linkage='complete').fit(moons)     

plt.figure(figsize=(10, 6))

plt.scatter(moons[:, 0], moons[:, 1], c=cluster.labels_);

In [ ]:
cluster=AC(linkage='average').fit(moons)     

plt.figure(figsize=(10, 6))

plt.scatter(moons[:, 0], moons[:, 1], c=cluster.labels_);

In [ ]:
cluster=AC(linkage='single').fit(moons)     

plt.figure(figsize=(10, 6))

plt.scatter(moons[:, 0], moons[:, 1], c=cluster.labels_);